In [1]:
import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

In [32]:
with connection.connect() as conns:
    df = conns.execute(text('SELECT * FROM "BIFIUSDT"')).mappings().all()
df = pd.DataFrame(df)
print(df)

      Precio   Simbolo                  Tiempo
0      362.4  BIFIUSDT 2023-06-10 04:45:39.640
1      355.7  BIFIUSDT 2023-06-10 04:45:44.332
2      356.6  BIFIUSDT 2023-06-10 04:45:45.619
3      356.9  BIFIUSDT 2023-06-10 04:45:46.196
4      357.1  BIFIUSDT 2023-06-10 04:45:46.846
...      ...       ...                     ...
8202   390.5  BIFIUSDT 2023-06-23 17:17:58.189
8203   390.5  BIFIUSDT 2023-06-23 17:24:14.052
8204   390.3  BIFIUSDT 2023-06-23 17:24:43.993
8205   390.1  BIFIUSDT 2023-06-23 17:25:46.354
8206   389.8  BIFIUSDT 2023-06-23 17:34:01.347

[8207 rows x 3 columns]


In [2]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/byday/6years/'
pairs = os.listdir(path)
pairs = pd.DataFrame(pairs,columns=['pairs'])

downs = pairs[pairs['pairs'].str.contains('DOWN')]
ups = pairs[pairs['pairs'].str.contains('UP')]
ups_downs = pd.concat([ups, downs], ignore_index=True)
ups_downs = ups_downs[ups_downs['pairs'].str.contains('SUPERUSDT')==False]

for x in ups_downs['pairs']:
    pairs = pairs[pairs['pairs'].str.contains(x)==False]

In [3]:
for pair in pairs['pairs']:
    with connection.connect() as conns:
        conns.execute(text("""ALTER TABLE "{pair}" ALTER COLUMN "Tiempo" TYPE TIMESTAMP USING("Tiempo"::timestamp)""".format(pair=pair)))

C:\Users\tinma\AppData\Local\Temp\ipykernel_21380\2270612637.py:3: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conns.execute(text("""ALTER TABLE "{pair}" ALTER COLUMN "Tiempo" TYPE TIMESTAMP USING("Tiempo"::timestamp)""".format(pair=pair)))


In [15]:

with connection.connect() as conns:
    df2 = conns.execute(text('SELECT table_name, column_name, data_type FROM information_schema.columns')).mappings().all()
df2 = pd.DataFrame(df2)
df2[df2['column_name']=='Tiempo']

,column_name,data_type,table_name
517,Tiempo,timestamp without time zone,DODOUSDT
518,Tiempo,timestamp without time zone,KSMUSDT
519,Tiempo,timestamp without time zone,RNDRUSDT
521,Tiempo,timestamp without time zone,BTTCUSDT
522,Tiempo,timestamp without time zone,THETAUSDT
...,...,...,...
1202,Tiempo,timestamp without time zone,CTSIUSDT
1203,Tiempo,timestamp without time zone,CVXUSDT
1204,Tiempo,timestamp without time zone,DARUSDT
1205,Tiempo,timestamp without time zone,LUNAUSDT


In [5]:
with connection.connect() as conns:
    df1 = conns.execute(text('SELECT * FROM "BTCUSDT"')).mappings().all()
df1 = pd.DataFrame(df1)
type(df1['Tiempo'][0])

pandas._libs.tslibs.timestamps.Timestamp